# Blatt 8

## Aufgabe 22: Fehlerfortpflanzung

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uncertainties.unumpy as unp
from uncertainties import ufloat

In [ ]:
sig0 =  0.2
sig1 =  0.2
rho  = -0.8
cov = [[sig0**2, rho * sig0 * sig1], [rho * sig0 * sig1, sig1**2]]

a = np.random.multivariate_normal(mean = [1, 1], cov = cov, size = 100)                                                  

In [ ]:
plt.scatter(a[:, 0], a[:, 1], alpha = 0.9, s = 5)
plt.xlabel('$a_0$')
plt.ylabel('$a_1$')

In [ ]:
x = np.linspace(-10, 10, 100)
stdy = [np.std(a[:, 0] + a[:, 1] * x_i) for x_i in x]   
plt.plot(x, stdy, label = 'Numerisch' )

def exact(x):
    return np.sqrt(sig0**2 + x**2 * sig1**2 + 2 * rho * sig0 * sig1 * x)
plt.plot(x, exact(x), label = 'Exakt')
plt.xlabel('$x$')
plt.ylabel('$\sigma_y$')
plt.legend()
plt.show()

In [ ]:
x = [-3, 0, 3]
for x_i in x:
    y = a[:, 0] + a[:, 1] * x_i
    print(f'x = {x_i}:\t y = {np.mean(y)} +/- {np.std(y)}')

Kommentar: Die numerischen Ergebnisse stimmen recht gut mit den exakt bestimmten Werten überein (siehe Handschriftliches). Die Präzision nimmt für große Beträge von $x$ ab (siehe Plot oben). Durch eine Vergrößerung des Samples könnte die Präzision gesteigert werden.

## Aufgabe 24: F(uckin')-Praktikum

In [ ]:
Asymmetrie = np.matrix([-0.032, 0.01, 0.057,
                       0.068, 0.076, 0.08, 
                       0.031, 0.005, -0.041, 
                       -0.09, -0.088, -0.074])
Psi = np.arange(0, 331, 30)

In [ ]:
A = np.matrix([np.cos(Psi / 180 * np.pi), np.sin(Psi / 180 * np.pi)]).T

In [ ]:
a = (A.T * A).I * A.T * Asymmetrie.T
a

In [ ]:
Psi_plot = np.linspace(0, 360, 100)
plt.plot(Psi, Asymmetrie.T, 'o', label = 'Daten')
plt.xlabel('$\Psi / °$')
plt.ylabel('Asymmetrie')
plt.plot(Psi_plot, 
         a[0, 0] * np.cos(Psi_plot / 180 * np.pi) + a[1,0] * np.sin(Psi_plot / 180 * np.pi), 
         label = 'Anpassung mit 1. Modell')
plt.legend()

In [ ]:
sigma = 0.011
Cov = np.array(sigma**2 * (A.T * A).I)

In [ ]:
rho = Cov[0, 1] / np.sqrt(Cov[0, 0] * Cov[1, 1])
print(f'a1 = {ufloat(a[0, 0], np.sqrt(Cov[0, 0]))}')
print(f'a2 = {ufloat(a[1, 0], np.sqrt(Cov[1, 1]))}')
print(f'Korrelationskoeffizient: {rho}')

Die Korrelation ist fast 0. Das Modell ist also gut gewählt, was letztlich an der Orthogoalität der Funktionen Sinus und Kosinus liegt.

In [ ]:
delta = unp.arctan(- a[1, 0] / a[0, 0])
A0 = a[0, 0] / unp.cos(unp.arctan(- a[1, 0] / a[0, 0]))

In [ ]:
a1 = a[0,0]
a2 = a[1,0]

J = np.matrix([[a1 / np.sqrt(a1**2 + a2**2), a2 / np.sqrt(a1**2 + a2**2)], 
     [a2 / (a1**2 + a2**2), -1 / (a1 + a2**2 / a1)]])
Cov2 = J * Cov * J.T
print(f'A_0 = {ufloat(A0, np.sqrt(Cov2[0, 0]))}')
print(f'delta = {ufloat(delta, np.sqrt(Cov2[1, 1]))}')
print(f'Korrelation: {Cov2[1, 0] / np.sqrt(Cov2[0, 0] * Cov2[1, 1]) }')

Die Korrelation ist ebenfalls sehr klein aber etwa 10x größer als bei dem anderen Modell. 

In [ ]:
Psi_plot = np.linspace(0, 360, 100)
plt.plot(Psi, Asymmetrie.T, 'o', label = 'Daten')
plt.xlabel('$\Psi / °$')
plt.ylabel('Asymmetrie')
plt.plot(Psi_plot, A0 * unp.cos(Psi_plot / 180 * np.pi + delta), label = 'Anpassung mit 2. Modell')
plt.legend()